In [ ]:
from IPython import display
import pandas as pd
from arcgis.gis import GIS

### Entitlement Management Task

####  Goal: For Each Entitlement, catalog each user that has that assignment.

#### Create a GIS as an Admin

In [ ]:
#
#  Update your profile/credentials here!
#
gis = GIS(profile='your_online_profile')

#### Access the License Manager

In [ ]:
lm = gis.admin.license

#### Get all the available licenses

In [ ]:
licenses = lm.all()

In [ ]:
gathered = []
for lic in licenses:
    try:
        stats = lic.all()
        for s in stats:
            s['entitlements'] = ",".join(s['entitlements'])
        gathered.extend(stats)
    except:
        print(lic)

In [ ]:
df = pd.DataFrame(gathered)

#### Create a Pandas DataFrame for Easy Visualization of Data

In [ ]:
groupby = pd.DataFrame(gathered).groupby('username')

In [ ]:
q = df.lastLogin == -1
df.loc[q, ['lastLogin']] = None
df.lastLogin = pd.to_datetime(df.lastLogin, unit='ms')

In [ ]:
exploded_df = (df
               .copy()
               .set_index(df.columns.drop('entitlements',1).tolist())
               .entitlements.str.split(',', expand=True)
               .stack()
               .reset_index()
               .rename(columns={0:'entitlements'})
               .loc[:, df.columns]
)

In [ ]:
exploded_df.head()

In [ ]:

grouped_df = exploded_df.groupby('entitlements')
for key, item in grouped_df:
    print(f"entitlement: {key}")
    display.display(grouped_df.get_group(key))

#### Total Entitlement and Extension Count

The groupby operation in Pandas allows for some quick analysis of licenses.  Here we get a count of all entitlements plus extensions using the `exploded_df` DataFrame.

In [ ]:
exploded_df.groupby('username').username.count()